In [1]:
from llama_index.llms.ollama import Ollama
import re
import json
import pandas as pd
import sys
import os
import numpy as np
import spacy
from heapq import nlargest
from llama_index.core import PromptTemplate
sys.path.append(os.path.abspath("MHQA-ontology"))
from prompts.prompt_templates import *
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Tuple, Union
from tqdm import tqdm

In [2]:
from sentence_transformers import SentenceTransformer

In [7]:
llm = Ollama(model="llama3.3", request_timeout= 280.0,keep_alive=-1)
response = llm.complete("What are the recurring archetypes in Bulgarian political myths?")
# print(response)

In [8]:
Hotpot_train = json.load(open('dataset/hotpot/hotpot_train_v1.1.json'))
# Hotpot_train = json.load(open('MHQA-ontology/dataset/hotpot/hotpot_train_v1.1.json'))

In [129]:
import importlib
import utils.RAG_process
importlib.reload(utils.RAG_process)
from utils.RAG_process import rag_generate

dataset = Hotpot_train[:10]

RAG_context = rag_generate(dataset,llm)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
/mnt/scratch/users/ts1201/MHQA-ontology/utils/RAG_process.py:521: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:

In [130]:
RAG_context

,Question,gold_answer,Supporting Facts,SPARQL query,Retrieval Facts,Retrieval Result,coverage_all,coverage_rate
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,"[[Arthur's Magazine, 0], [First for Women, 0]]",SELECT ?magazine ?start_date WHERE { { Arthur'...,"[[Radio City (Indian radio station), 4], [Arth...","[{'title': 'First for Women', 'paragraphs': ['...",1,0.152174
1,The Oberoi family is part of a hotel company t...,Delhi,"[[Oberoi family, 0], [The Oberoi Group, 0]]",SELECT ?city WHERE { The_Oberoi_Family part_of...,"[[Hotel Tallcorn, 0], [The Oberoi Group, 1], [...","[{'title': 'Oberoi family', 'paragraphs': ['Th...",1,0.282051
2,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,"[[Allie Goertz, 0], [Allie Goertz, 1], [Allie ...",SELECT ?inspiration WHERE { Matt_Groening name...,"[[Homer Simpson, 4], [List of The Simpsons vid...","[{'title': 'Lisa Simpson', 'paragraphs': [' Ca...",0,0.147059
3,What nationality was James Henry Miller's wife?,American,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",SELECT ?nationality WHERE { James_Henry_Miller...,"[[Ewan MacColl, 0], [June Miller, 0], [Incest:...","[{'title': 'Jim Miller (Australian footballer,...",1,0.434783
4,Cadmium Chloride is slightly soluble in this c...,alcohol,"[[Cadmium chloride, 1], [Ethanol, 0]]",SELECT ?chemical ?alternative_name WHERE { Cad...,"[[Chloride, 2], [Water blue, 0], [Water blue, ...","[{'title': 'Cadmium chloride', 'paragraphs': [...",0,0.274510
5,Which tennis player won more Grand Slam titles...,Jonathan Stark,"[[Jonathan Stark (tennis), 0], [Jonathan Stark...",SELECT ?leconte_titles ?stark_titles WHERE { H...,"[[Steffi Graf, 4], [Steffi Graf, 1], [1986 Gra...","[{'title': 'Henri Leconte', 'paragraphs': [' L...",0,0.163934
6,Which genus of moth in the world's seventh-lar...,Crambidae,"[[Indogrammodes, 0], [Indogrammodes, 1], [Indi...",SELECT ?genus WHERE { ?country is_seventh_larg...,"[[Eumacaria, 0], [Indogrammodes, 0], [Eutrapel...","[{'title': 'Geography of India', 'paragraphs':...",0,0.562500
7,Who was once considered the best kick boxer in...,Badr Hari,"[[Global Fighting Championship, 1], [Global Fi...",SELECT ?person WHERE { ?person is_best_kickbox...,"[[Badr Hari, 1], [Outrageous Betrayal, 2], [Pr...","[{'title': 'Badr Hari', 'paragraphs': [' Hari ...",0,0.400000
8,"The Dutch-Belgian television series that ""Hous...",2006,"[[House of Anubis, 0], [Het Huis Anubis, 1]]","SELECT ?year WHERE { ?series rdfs:label ""House...","[[Nathalia Ramos, 1], [House of Anubis, 1], [W...","[{'title': 'House of Anubis', 'paragraphs': ['...",1,0.320755
9,What is the length of the track where the 2013...,6.213 km long,"[[2013 Liqui Moly Bathurst 12 Hour, 0], [2013 ...",SELECT ?track_length WHERE { EVENT:2013_Liqui_...,"[[Mount Panorama Circuit, 2], [2017 Interconti...","[{'title': '2013 Liqui Moly Bathurst 12 Hour',...",1,0.379310


In [131]:
dataset_df = pd.DataFrame(dataset)
refine_dataset = dataset_df[dataset_df['question'].isin(RAG_context['Question'])].reset_index(drop=True)
refine_dataset

,supporting_facts,level,question,context,answer,_id,type
0,"[[Arthur's Magazine, 0], [First for Women, 0]]",medium,Which magazine was started first Arthur's Maga...,"[[Radio City (Indian radio station), [Radio Ci...",Arthur's Magazine,5a7a06935542990198eaf050,comparison
1,"[[Oberoi family, 0], [The Oberoi Group, 0]]",medium,The Oberoi family is part of a hotel company t...,"[[Ritz-Carlton Jakarta, [The Ritz-Carlton Jaka...",Delhi,5a879ab05542996e4f30887e,bridge
2,"[[Allie Goertz, 0], [Allie Goertz, 1], [Allie ...",hard,Musician and satirist Allie Goertz wrote a son...,"[[Lisa Simpson, [Lisa Marie Simpson is a ficti...",President Richard Nixon,5a8d7341554299441c6b9fe5,bridge
3,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",medium,What nationality was James Henry Miller's wife?,"[[Moloch: or, This Gentile World, [Moloch: or,...",American,5a82171f5542990a1d231f4a,bridge
4,"[[Cadmium chloride, 1], [Ethanol, 0]]",medium,Cadmium Chloride is slightly soluble in this c...,"[[Cadmium chloride, [Cadmium chloride is a whi...",alcohol,5a84dd955542997b5ce3ff79,bridge
5,"[[Jonathan Stark (tennis), 0], [Jonathan Stark...",medium,Which tennis player won more Grand Slam titles...,"[[Li Na, [Li Na (; ; born 26 February 1982) is...",Jonathan Stark,5a7e36045542991319bc9440,comparison
6,"[[Indogrammodes, 0], [Indogrammodes, 1], [Indi...",hard,Which genus of moth in the world's seventh-lar...,"[[India, [India, officially the Republic of In...",Crambidae,5adf44985542993a75d2646d,bridge
7,"[[Global Fighting Championship, 1], [Global Fi...",easy,Who was once considered the best kick boxer in...,"[[Verano de Escándalo (1998), [The 1998 Verano...",Badr Hari,5a832c3455429954d2e2ec41,bridge
8,"[[House of Anubis, 0], [Het Huis Anubis, 1]]",medium,"The Dutch-Belgian television series that ""Hous...","[[House of Anubis, [House of Anubis is a myste...",2006,5a7d0db955429909bec76924,bridge
9,"[[2013 Liqui Moly Bathurst 12 Hour, 0], [2013 ...",hard,What is the length of the track where the 2013...,"[[Mount Panorama Circuit, [Mount Panorama Circ...",6.213 km long,5a89372855429951533612e6,bridge


In [136]:
import importlib
import utils.RAG_process
importlib.reload(utils.RAG_process)
from utils.RAG_process import refine_supporting_facts
# Example usage:

# Process with verbose output
refined_df = refine_supporting_facts(RAG_context, RAG_context, refine_dataset)

# Check results for a specific row
idx = 1
print(f"Original facts: {RAG_context['Retrieval Facts'][idx]}")
print(f"Refined facts: {refined_df['Retrieval Facts'][idx]}")


INFO:utils.RAG_process:Processed 0 rows...
INFO:utils.RAG_process:Processing complete


Original facts: [['Hotel Tallcorn', 0], ['The Oberoi Group', 1], ['Ritz-Carlton Jakarta', 1], ['289th Military Police Company', 1], ['The Oberoi Group', 0], ['Glennwanis Hotel', 1], ['Oberoi family', 0], ['Glennwanis Hotel', 4], ['Hotel Tallcorn', 4], ['Future Fibre Technologies', 0], ['Mohan Singh Oberoi', 0]]
Refined facts: [['Hotel Tallcorn', 4], ['The Oberoi Group', 0], ['Ritz-Carlton Jakarta', 1], ['289th Military Police Company', 1], ['The Oberoi Group', 1], ['Glennwanis Hotel', 4], ['Oberoi family', 0], ['Glennwanis Hotel', 4], ['Hotel Tallcorn', 4], ['Future Fibre Technologies', 0], ['Mohan Singh Oberoi', 0]]


In [137]:
import ast

def compare_facts(refined_df):
    """
    Compares if all Supporting Facts are in Retrieval Facts.
    Sets coverage_all to 1 if all are present, else 0.
    """
    df = refined_df.copy()
    
    # Function to check if all supporting facts are in retrieval facts
    def check_coverage(row):
        supporting = row['Supporting Facts']
        retrieval = row['Retrieval Facts']
        
        # Convert strings to lists if needed
        if isinstance(supporting, str):
            supporting = ast.literal_eval(supporting)
        if isinstance(retrieval, str):
            retrieval = ast.literal_eval(retrieval)
        
        # Convert lists to strings for simple comparison
        supporting_str = [str(item) for item in supporting]
        retrieval_str = [str(item) for item in retrieval]
        
        # Check if all supporting facts are in retrieval facts
        all_present = all(item in retrieval_str for item in supporting_str)
        return 1 if all_present else 0
    
    # Apply the function to each row
    df['coverage_all'] = df.apply(check_coverage, axis=1)
    
    return df

refined_df2 = compare_facts(refined_df)


In [140]:
refined_df2

,Question,gold_answer,Supporting Facts,SPARQL query,Retrieval Facts,Retrieval Result,coverage_all,coverage_rate
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,"[[Arthur's Magazine, 0], [First for Women, 0]]",SELECT ?magazine ?start_date WHERE { { Arthur'...,"[[Radio City (Indian radio station), 4], [Arth...","[{'title': 'First for Women', 'paragraphs': ['...",1,0.152174
1,The Oberoi family is part of a hotel company t...,Delhi,"[[Oberoi family, 0], [The Oberoi Group, 0]]",SELECT ?city WHERE { The_Oberoi_Family part_of...,"[[Hotel Tallcorn, 4], [The Oberoi Group, 0], [...","[{'title': 'Oberoi family', 'paragraphs': ['Th...",1,0.282051
2,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,"[[Allie Goertz, 0], [Allie Goertz, 1], [Allie ...",SELECT ?inspiration WHERE { Matt_Groening name...,"[[Homer Simpson, 4], [List of The Simpsons vid...","[{'title': 'Lisa Simpson', 'paragraphs': [' Ca...",0,0.147059
3,What nationality was James Henry Miller's wife?,American,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",SELECT ?nationality WHERE { James_Henry_Miller...,"[[Ewan MacColl, 0], [June Miller, 0], [Incest:...","[{'title': 'Jim Miller (Australian footballer,...",1,0.434783
4,Cadmium Chloride is slightly soluble in this c...,alcohol,"[[Cadmium chloride, 1], [Ethanol, 0]]",SELECT ?chemical ?alternative_name WHERE { Cad...,"[[Chloride, 4], [Water blue, 2], [Water blue, ...","[{'title': 'Cadmium chloride', 'paragraphs': [...",0,0.274510
5,Which tennis player won more Grand Slam titles...,Jonathan Stark,"[[Jonathan Stark (tennis), 0], [Jonathan Stark...",SELECT ?leconte_titles ?stark_titles WHERE { H...,"[[Steffi Graf, 4], [Steffi Graf, 1], [1986 Gra...","[{'title': 'Henri Leconte', 'paragraphs': [' L...",0,0.163934
6,Which genus of moth in the world's seventh-lar...,Crambidae,"[[Indogrammodes, 0], [Indogrammodes, 1], [Indi...",SELECT ?genus WHERE { ?country is_seventh_larg...,"[[Eumacaria, 1], [Indogrammodes, 0], [Eutrapel...","[{'title': 'Geography of India', 'paragraphs':...",0,0.562500
7,Who was once considered the best kick boxer in...,Badr Hari,"[[Global Fighting Championship, 1], [Global Fi...",SELECT ?person WHERE { ?person is_best_kickbox...,"[[Badr Hari, 1], [Outrageous Betrayal, 2], [Pr...","[{'title': 'Badr Hari', 'paragraphs': [' Hari ...",0,0.400000
8,"The Dutch-Belgian television series that ""Hous...",2006,"[[House of Anubis, 0], [Het Huis Anubis, 1]]","SELECT ?year WHERE { ?series rdfs:label ""House...","[[Nathalia Ramos, 1], [House of Anubis, 1], [W...","[{'title': 'House of Anubis', 'paragraphs': ['...",1,0.320755
9,What is the length of the track where the 2013...,6.213 km long,"[[2013 Liqui Moly Bathurst 12 Hour, 0], [2013 ...",SELECT ?track_length WHERE { EVENT:2013_Liqui_...,"[[Mount Panorama Circuit, 2], [2017 Interconti...","[{'title': '2013 Liqui Moly Bathurst 12 Hour',...",0,0.379310


In [142]:
RAG_context_1 = refined_df2[refined_df2['coverage_all'] == 1].reset_index(drop=True)
RAG_context_1

,Question,gold_answer,Supporting Facts,SPARQL query,Retrieval Facts,Retrieval Result,coverage_all,coverage_rate
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,"[[Arthur's Magazine, 0], [First for Women, 0]]",SELECT ?magazine ?start_date WHERE { { Arthur'...,"[[Radio City (Indian radio station), 4], [Arth...","[{'title': 'First for Women', 'paragraphs': ['...",1,0.152174
1,The Oberoi family is part of a hotel company t...,Delhi,"[[Oberoi family, 0], [The Oberoi Group, 0]]",SELECT ?city WHERE { The_Oberoi_Family part_of...,"[[Hotel Tallcorn, 4], [The Oberoi Group, 0], [...","[{'title': 'Oberoi family', 'paragraphs': ['Th...",1,0.282051
2,What nationality was James Henry Miller's wife?,American,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",SELECT ?nationality WHERE { James_Henry_Miller...,"[[Ewan MacColl, 0], [June Miller, 0], [Incest:...","[{'title': 'Jim Miller (Australian footballer,...",1,0.434783
3,"The Dutch-Belgian television series that ""Hous...",2006,"[[House of Anubis, 0], [Het Huis Anubis, 1]]","SELECT ?year WHERE { ?series rdfs:label ""House...","[[Nathalia Ramos, 1], [House of Anubis, 1], [W...","[{'title': 'House of Anubis', 'paragraphs': ['...",1,0.320755


In [ ]:
from utils.KG_builder import build_rdflib_knowledge_graph,process_contexts_in_batches

index = 1
all_facts = RAG_context_1['Retrieval Result'][index]

question = RAG_context_1['Question'][index]
context_data = RAG_context_1['Retrieval Result'][index]




knowledge_graph = process_contexts_in_batches(
    context_items=all_facts,
    llm=llm,
    prompt_template=prompt_template_context,
    batch_size=2  # Process 2 contexts at a time
)

kg = build_rdflib_knowledge_graph(knowledge_graph)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [ ]:
import importlib
import ultis.iterative_reasoner
importlib.reload(ultis.iterative_reasoner)
from ultis.iterative_reasoner import process_question_with_kg_awareness



# Process the question with KG-aware exploration
result = process_question_with_kg_awareness(
    kg=kg,  
    context_data=context_data, 
    question=question,
    llm=llm,
    max_iterations=4  
)

# Display the concise answer
print(f"Question: {result['question']}")
print(f"Answer: {result['answer']}")  # Now returns just "Delhi" instead of a sentence
print(f"Evidence source: {result['evidence_source']}")

Processing question: The Oberoi family is part of a hotel company that has a head office in what city?
Exploring knowledge graph for relevant entities...
Found 3 entities related to the question
Available predicates: announced_renovation_plans_for, associated_with, birth_date, category, countries_of_operation...
Available entity types: Location, Organization, apartment, building, company...
Generating initial query with KG awareness...


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Initial query:
PREFIX kg: <http://example.org/kg/>

SELECT ?city 
WHERE {
  ?company kg:name "The Oberoi Group" .
  ?company kg:hasType kg:company .
  ?family kg:name "Oberoi family" .
  ?family kg:associated_with ?company .
  ?hotel kg:name "Oberoi Hotels & Resorts" .
  ?hotel kg:hasType kg:hotel_company .
  ?hotel kg:operates_in ?city .
}
Initial query success: True


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



Final answer: Delhi
Evidence source: Knowledge Graph
Confidence: 0.9
Question: The Oberoi family is part of a hotel company that has a head office in what city?
Answer: Delhi
Evidence source: Knowledge Graph


In [53]:
result['answer']    

'Delhi'

In [51]:
result['full_response_for_final']    

'## Step 1: Analyze what information we have from the KG exploration\nThe entities found in the KG include The Oberoi Group, Oberoi Hotels & Resorts, and the Oberoi family. The relationships between these entities indicate that the Oberoi family is associated with The Oberoi Group, which has a type of company, and Oberoi Hotels & Resorts is an organization with types hotel_company and organization. This relates to the question because it identifies key components of the Oberoi family\'s involvement in hotels.\n\n## Step 2: Identify relevant information in the context\nThe parts of the context that mention entities from the KG exploration include Source 1, which mentions the Oberoi family\'s involvement in hotels through The Oberoi Group, and Source 2, which specifically states that The Oberoi Group is a hotel company with its head office in Delhi. Additionally, Source 3 provides more information about Mohan Singh Oberoi, the founder of Oberoi Hotels & Resorts, but does not directly men

SELECT ?head_office_in 
WHERE { 
    ?company kg:name "The Oberoi Group" .
    ?company kg:is_a "hotel_company" .
    ?company kg:has_head_office_in ?head_office_in .
}

In [ ]:
query2 = """
PREFIX kg: <http://example.org/kg/>

SELECT ?s
WHERE {
    ?s kg:name "Richard Nixon" .
}
"""
#   ?o kg:name "Milhouse" .
#   ?s kg:created_by ?o .
#   ?o kg:named_after ?named_after .
# Execute the query
for row in kg.query(query2):
    print(row)

'question': 'Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?',

['Allie Goertz',
   ['Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician.',
    ' Goertz is known for her satirical songs based on various pop culture topics.',
    ' Her videos are posted on YouTube under the name of Cossbysweater.',

['Milhouse Van Houten',
   ['Milhouse Mussolini van Houten is a fictional character featured in the animated television series "The Simpsons", voiced by Pamela Hayden, and created by Matt Groening who named the character after President Richard Nixon\'s middle name.',


In [ ]:
Hotpot_train[2]

{'supporting_facts': [['Allie Goertz', 0],
  ['Allie Goertz', 1],
  ['Allie Goertz', 2],
  ['Milhouse Van Houten', 0]],
 'level': 'hard',
 'question': 'Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?',
 'context': [['Lisa Simpson',
   ['Lisa Marie Simpson is a fictional character in the animated television series "The Simpsons".',
    ' She is the middle child and most intelligent of the Simpson family.',
    ' Voiced by Yeardley Smith, Lisa first appeared on television in "The Tracey Ullman Show" short "Good Night" on April 19, 1987.',
    ' Cartoonist Matt Groening created and designed her while waiting to meet James L. Brooks.',
    ' Groening had been invited to pitch a series of shorts based on his comic "Life in Hell", but instead decided to create a new set of characters.',
    ' He named the elder Simpson daughter after his younger sister Lisa Groening.',
    ' After appearing on "The Tracey Ullman 